In [1]:
import pyodbc
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

conn = pyodbc.connect(
    r'DRIVER={ODBC Driver 17 for SQL Server};'
    r'SERVER=(localdb)\MSSQLLocalDB;'
    r'DATABASE=AcademiaDW;'
    r'Trusted_Connection=yes;'
)


InterfaceError: ('28000', '[28000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Login failed for user \'DESKTOP-U623P07\\Matt\'. (18456) (SQLDriverConnect); [28000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Cannot open database "AcademiaDW" requested by the login. The login failed. (4060); [28000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Login failed for user \'DESKTOP-U623P07\\Matt\'. (18456); [28000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Cannot open database "AcademiaDW" requested by the login. The login failed. (4060)')

In [ ]:
#Formatação paras as colunas nas querys.
def executar_query(sql, formatacoes=None):
    df = pd.read_sql(sql, conn)

    if formatacoes:
        for col, tipo in formatacoes.items():
            if col in df.columns:
                if tipo == "int":
                    df[col] = df[col].fillna(0).astype(int).map('{:,}'.format).str.replace(',', '.')
                elif tipo == "float":
                    df[col] = df[col].fillna(0).round(2).map(lambda x: f"{x:,.2f}".replace(",", "X").replace(".", ",").replace("X", "."))
                elif tipo == "money":
                    df[col] = df[col].fillna(0).round(2).map(lambda x: f"R$ {x:,.2f}".replace(",", "X").replace(".", ",").replace("X", "."))
                elif tipo == "decimal":
                    df[col] = df[col].fillna(0).round(4).map(lambda x: f"{x:,.4f}".replace(",", "X").replace(".", ",").replace("X", "."))
                elif tipo == "percent":
                    df[col] = df[col].fillna(0).round(2).map(lambda x: f"{x:.2%}".replace(".", ","))
    return df


In [ ]:
sql_1 = """
SELECT status, COUNT(*) AS total_alunos
FROM dim_usuario
GROUP BY status;
"""
df_1 = executar_query(sql_1, {"total_alunos": "int"})
print(df_1)

      status total_alunos
0  Cancelado          645
1      Ativo          689
2    Inativo          666


In [ ]:
sql_2 = """
SELECT AVG(total_presencas) AS freq_media_semanal
FROM fato_frequencia;
"""
df_2 = executar_query(sql_2, {"freq_media_semanal": "float"})
print(df_2)


  freq_media_semanal
0               1,00


In [ ]:
sql_3 = """
SELECT semanas_presentes, COUNT(id_usuario) AS qtd_alunos
FROM fato_final_aluno
GROUP BY semanas_presentes
ORDER BY semanas_presentes DESC;
"""
df_3 = executar_query(sql_3, {"semanas_presentes": "int", "qtd_alunos": "int"})
print(df_3)


   semanas_presentes qtd_alunos
0                 24          1
1                 23          1
2                 22          3
3                 21          1
4                 20          6
5                 19         13
6                 18         14
7                 17         27
8                 16         54
9                 15         79
10                14        126
11                13        163
12                12        210
13                11        250
14                10        249
15                 9        238
16                 8        209
17                 7        154
18                 6        106
19                 5         54
20                 4         30
21                 3          6
22                 2          5
23                 1          1


In [ ]:
sql_4 = """
SELECT id_usuario, media_nota
FROM fato_final_aluno
ORDER BY media_nota DESC;
"""
df_4 = executar_query(sql_4, {"id_usuario": "int", "media_nota": "float"})
print(df_4.head(10))


  id_usuario media_nota
0         54       5,00
1        108       5,00
2        179       5,00
3        350       5,00
4        464       5,00
5        735       5,00
6        886       5,00
7        964       5,00
8        968       5,00
9      1.196       5,00


In [ ]:
sql_5 = """
SELECT id_usuario, perda_peso
FROM fato_final_aluno
ORDER BY perda_peso DESC
OFFSET 0 ROWS FETCH NEXT 10 ROWS ONLY;
"""
df_5 = executar_query(sql_5, {"id_usuario": "int", "perda_peso": "float"})
print(df_5)


  id_usuario perda_peso
0      1.793      68,50
1        356      68,30
2        572      66,80
3      1.940      66,40
4      1.398      66,00
5        253      65,50
6      1.078      65,00
7        138      63,90
8      1.241      63,80
9        707      63,50


In [ ]:
sql_6 = """
SELECT
  CASE 
    WHEN idade BETWEEN 18 AND 25 THEN '18-25'
    WHEN idade BETWEEN 26 AND 35 THEN '26-35'
    WHEN idade BETWEEN 36 AND 45 THEN '36-45'
    WHEN idade BETWEEN 46 AND 55 THEN '46-55'
    ELSE '56+' 
  END AS faixa_idade,
  AVG(ganho_forca) AS ganho_medio_forca
FROM fato_final_aluno
GROUP BY 
  CASE 
    WHEN idade BETWEEN 18 AND 25 THEN '18-25'
    WHEN idade BETWEEN 26 AND 35 THEN '26-35'
    WHEN idade BETWEEN 36 AND 45 THEN '36-45'
    WHEN idade BETWEEN 46 AND 55 THEN '46-55'
    ELSE '56+' 
  END
ORDER BY faixa_idade;
"""
df_6 = executar_query(sql_6, {"ganho_medio_forca": "float"})
print(df_6)


  faixa_idade ganho_medio_forca
0       18-25              0,50
1       26-35             -4,59
2       36-45             10,38
3       46-55             -1,27
4         56+              5,14


In [ ]:
sql_7 = """
SELECT FORMAT(data_inicio, 'yyyy-MM') AS mes_ano,
       COUNT(CASE WHEN status IN ('Inativo', 'Cancelado') THEN 1 END) AS alunos_evasao,
       COUNT(*) AS total_alunos,
       CAST(COUNT(CASE WHEN status IN ('Inativo', 'Cancelado') THEN 1 END) * 1.0 / COUNT(*) AS DECIMAL(5,4)) AS taxa_evasao
FROM dim_usuario
GROUP BY FORMAT(data_inicio, 'yyyy-MM')
ORDER BY mes_ano;
"""
df_7 = executar_query(sql_7, {"alunos_evasao": "int", "total_alunos": "int", "taxa_evasao": "decimal"})
print(df_7)


    mes_ano alunos_evasao total_alunos taxa_evasao
0   2022-07            34           55      0,6182
1   2022-08            56          100      0,5600
2   2022-09            52           82      0,6341
3   2022-10            41           71      0,5775
4   2022-11            46           70      0,6571
5   2022-12            53           84      0,6310
6   2023-01            54           82      0,6585
7   2023-02            47           80      0,5875
8   2023-03            37           53      0,6981
9   2023-04            53           78      0,6795
10  2023-05            67           92      0,7283
11  2023-06            49           84      0,5833
12  2023-07            67           89      0,7528
13  2023-08            56           81      0,6914
14  2023-09            61           82      0,7439
15  2023-10            49           75      0,6533
16  2023-11            51           78      0,6538
17  2023-12            75          106      0,7075
18  2024-01            55      

In [ ]:
sql_8 = """
SELECT id_usuario, total_feedbacks
FROM fato_final_aluno
ORDER BY total_feedbacks DESC
OFFSET 0 ROWS FETCH NEXT 10 ROWS ONLY;
"""
df_8 = executar_query(sql_8, {"id_usuario": "int", "total_feedbacks": "int"})
print(df_8)


  id_usuario total_feedbacks
0      1.587              15
1      1.924              15
2        316              14
3        397              14
4      1.953              14
5      1.998              14
6         70              13
7        202              13
8        278              13
9        463              13


In [ ]:
sql_9 = """
SELECT semanas_presentes, AVG(media_nota) AS media_feedback
FROM fato_final_aluno
GROUP BY semanas_presentes
ORDER BY semanas_presentes DESC;
"""
df_9 = executar_query(sql_9, {"semanas_presentes": "int", "media_feedback": "float"})
print(df_9)


   semanas_presentes media_feedback
0                 24           2,88
1                 23           3,11
2                 22           2,26
3                 21           2,75
4                 20           3,02
5                 19           3,18
6                 18           3,43
7                 17           3,23
8                 16           2,88
9                 15           3,14
10                14           3,03
11                13           3,06
12                12           2,91
13                11           3,00
14                10           2,95
15                 9           2,98
16                 8           3,02
17                 7           2,98
18                 6           2,94
19                 5           2,98
20                 4           2,90
21                 3           2,90
22                 2           2,55
23                 1           1,83


In [ ]:
sql_10 = """
SELECT genero, AVG(perda_peso) AS media_perda_peso
FROM fato_final_aluno
GROUP BY genero;
"""
df_10 = executar_query(sql_10, {"media_perda_peso": "float"})
print(df_10)


      genero media_perda_peso
0   Feminino             0,81
1  Masculino            -0,34


In [ ]:
sql_11 = """
SELECT status, AVG(ganho_forca) AS media_ganho_forca
FROM fato_final_aluno
GROUP BY status;
"""
df_11 = executar_query(sql_11, {"media_ganho_forca": "float"})
print(df_11)


      status media_ganho_forca
0  Cancelado              0,10
1      Ativo             -4,06
2    Inativo             10,68


In [ ]:
sql_12 = """
SELECT 
    CAST(SUM(CASE WHEN semanas_presentes > 10 THEN 1 ELSE 0 END) * 100.0 / COUNT(*) AS DECIMAL(5,2)) AS percentual_aderencia_alta
FROM fato_final_aluno;
"""
df_12 = executar_query(sql_12, {"percentual_aderencia_alta": "float"})
print(df_12)


  percentual_aderencia_alta
0                     47,40
